In [1]:
import sys

sys.path.append('/user/ms5941/NLP')

from config import *
from utilities import *

import gensim

import glob
import os
import re

import numpy as np
import pandas as pd

import json

/user/ms5941/.local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from gensim.models import ldaseqmodel

In [3]:
THEME = 'Volatility'


In [4]:
# Load dictionary and corpus
dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (THEME, THEME))

2021-04-25 22:52:07,540 : INFO : loading Dictionary object from /work/ms5941/NLP/Temp/Volatility/Volatility_less_restricted.dict
2021-04-25 22:52:07,546 : INFO : Dictionary lifecycle event {'fname': '/work/ms5941/NLP/Temp/Volatility/Volatility_less_restricted.dict', 'datetime': '2021-04-25T22:52:07.545390', 'gensim': '4.0.1', 'python': '3.7.7 (default, May  7 2020, 21:25:33) \n[GCC 7.3.0]', 'platform': 'Linux-4.19.0-16-amd64-x86_64-with-debian-10.9', 'event': 'loaded'}


In [5]:
# 1. Look for ids of related words to theme in the dictionary. This will be a list of ids for each year.
# 

In [6]:
words = pd.read_csv(TEMP_PATH + '/%s_words.csv' % THEME.lower(), index_col=0)

In [7]:
year_name_list = dict()
year_id_list = dict()
for col in words.columns:
    reqd_words = list(words[col].values) + [THEME.lower()]
    reqd_ids = [k for k,v in dictionary_all.items() if v in reqd_words]
    year_name_list[col] = [dictionary_all[i] for i in reqd_ids]
    year_id_list[col] = reqd_ids

In [8]:
year_name_list

{'1996': ['volatility', 'downside', 'upside', 'trend'],
 '1997': ['volatility', 'options'],
 '1998': ['volatility', 'options', 'activity'],
 '1999': ['weakness', 'volatility', 'options'],
 '2000': ['weakness', 'volatility'],
 '2001': ['volatility', 'uncertainty'],
 '2002': ['weakness', 'volatility', 'volatile', 'uncertainty', 'strength'],
 '2003': ['weakness',
  'volatility',
  'volatile',
  'declines',
  'uncertainty',
  'exposure'],
 '2004': ['weakness', 'volatility', 'risk', 'exposure'],
 '2005': ['weakness',
  'volatility',
  'volatile',
  'risk',
  'uncertainty',
  'exposure'],
 '2006': ['weakness',
  'volatility',
  'risk',
  'uncertainty',
  'exposure',
  'upside'],
 '2007': ['weakness', 'volatility', 'uncertainty', 'upside'],
 '2008': ['volatility', 'volatile', 'upside'],
 '2009': ['volatility', 'downside', 'upside'],
 '2010': ['volatility', 'volatile', 'downside', 'upside'],
 '2011': ['weakness', 'volatility', 'volatile', 'uncertainty'],
 '2012': ['volatility', 'volatile', 'do

In [9]:
years = [str(year) for year in range(START_YEAR, END_YEAR + 1)]

In [10]:
good_words_count_per_year = dict()
for year in years:
    good_words_count_per_year[year] = []
    with open(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year)) as f:
        a = json.load(f)
        for article in a:
            good_words_count_per_year[year].append(len([word for word in article if word in year_name_list[year]]))
    print(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year), 'done!')

/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_1996.json done!
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_1997.json done!
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_1998.json done!
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_1999.json done!
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_2000.json done!
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_2001.json done!
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_2002.json done!
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_2003.json done!
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_2004.json done!
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_2005.json done!
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_2006.json done!
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_2007.json done!
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles

In [11]:
all_tok_articles = []
yearly_slices = []
for year in years:
    # Get the 1% threshold count (the top 1% of most relevant articles)
    x = good_words_count_per_year[year]
    x.sort()
    threshold = x[-int(len(x)*0.01):][0]
    year_article_count = 0
    with open(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year)) as f:
        all_articles = json.load(f)
        article_index = 0
        for articles in all_articles:
            if good_words_count_per_year[year][article_index] >= threshold:
                all_tok_articles.append(articles)
                year_article_count += 1    
            article_index += 1
    yearly_slices.append(year_article_count)
    print(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year), 'done!')
    print('Count for the year:', year_article_count, 'Vs original:', len(all_articles))

# dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (theme, theme))


/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_1996.json done!
Count for the year: 133 Vs original: 12537
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_1997.json done!
Count for the year: 185 Vs original: 17826
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_1998.json done!
Count for the year: 380 Vs original: 38090
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_1999.json done!
Count for the year: 260 Vs original: 23782
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_2000.json done!
Count for the year: 229 Vs original: 20482
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_2001.json done!
Count for the year: 306 Vs original: 17410
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_2002.json done!
Count for the year: 405 Vs original: 38270
/work/ms5941/NLP/Temp/Volatility/Volatility_Articles_Tokenized_2003.json done!
Count for the year: 447 Vs original: 35783
/work/ms5941/NLP/Temp/Vo

In [12]:
dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (theme, theme))

class MyCorpus:
    def __iter__(self):
        for doc in all_tok_articles:
            yield dictionary_all.doc2bow(doc)

corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)

gensim.corpora.MmCorpus.serialize(TEMP_PATH + '/%s/%s_less_restricted.mm' % (theme, theme), corpus_memory_friendly)


2021-04-25 22:59:26,713 : INFO : loading Dictionary object from /work/ms5941/NLP/Temp/Volatility/Volatility_less_restricted.dict
2021-04-25 22:59:26,717 : INFO : Dictionary lifecycle event {'fname': '/work/ms5941/NLP/Temp/Volatility/Volatility_less_restricted.dict', 'datetime': '2021-04-25T22:59:26.717430', 'gensim': '4.0.1', 'python': '3.7.7 (default, May  7 2020, 21:25:33) \n[GCC 7.3.0]', 'platform': 'Linux-4.19.0-16-amd64-x86_64-with-debian-10.9', 'event': 'loaded'}
2021-04-25 22:59:26,719 : INFO : storing corpus in Matrix Market format to /work/ms5941/NLP/Temp/Volatility/Volatility_less_restricted.mm
2021-04-25 22:59:26,729 : INFO : saving sparse matrix to /work/ms5941/NLP/Temp/Volatility/Volatility_less_restricted.mm
2021-04-25 22:59:26,730 : INFO : PROGRESS: saving document #0


2021-04-25 22:59:27,131 : INFO : PROGRESS: saving document #1000
2021-04-25 22:59:27,720 : INFO : PROGRESS: saving document #2000
2021-04-25 22:59:28,323 : INFO : PROGRESS: saving document #3000
2021-04-25 22:59:28,984 : INFO : PROGRESS: saving document #4000
2021-04-25 22:59:29,584 : INFO : PROGRESS: saving document #5000
2021-04-25 22:59:30,205 : INFO : PROGRESS: saving document #6000
2021-04-25 22:59:30,880 : INFO : PROGRESS: saving document #7000
2021-04-25 22:59:31,575 : INFO : PROGRESS: saving document #8000
2021-04-25 22:59:32,230 : INFO : PROGRESS: saving document #9000
2021-04-25 22:59:32,980 : INFO : PROGRESS: saving document #10000
2021-04-25 22:59:33,511 : INFO : PROGRESS: saving document #11000
2021-04-25 22:59:34,246 : INFO : PROGRESS: saving document #12000
2021-04-25 22:59:34,956 : INFO : PROGRESS: saving document #13000
2021-04-25 22:59:35,492 : INFO : PROGRESS: saving document #14000
2021-04-25 22:59:36,217 : INFO : PROGRESS: saving document #15000
2021-04-25 22:59:36

In [13]:
sum(yearly_slices)

15234

In [4]:
with open(TEMP_PATH + '/%s/%s_yearly_slices.txt' % (THEME, THEME), 'r') as f:
    yearly_slices = json.load(f)

In [5]:
dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (THEME, THEME))
corpus_all = gensim.corpora.MmCorpus(TEMP_PATH + '/%s/%s_less_restricted.mm' % (THEME, THEME))


2021-04-26 17:24:40,715 : INFO : loading Dictionary object from /work/ms5941/NLP/Temp/Volatility/Volatility_less_restricted.dict
2021-04-26 17:24:40,718 : INFO : Dictionary lifecycle event {'fname': '/work/ms5941/NLP/Temp/Volatility/Volatility_less_restricted.dict', 'datetime': '2021-04-26T17:24:40.718782', 'gensim': '4.0.1', 'python': '3.7.7 (default, May  7 2020, 21:25:33) \n[GCC 7.3.0]', 'platform': 'Linux-4.19.0-16-amd64-x86_64-with-debian-10.9', 'event': 'loaded'}
2021-04-26 17:24:40,721 : INFO : loaded corpus index from /work/ms5941/NLP/Temp/Volatility/Volatility_less_restricted.mm.index
2021-04-26 17:24:40,723 : INFO : initializing cython corpus reader from /work/ms5941/NLP/Temp/Volatility/Volatility_less_restricted.mm
2021-04-26 17:24:40,724 : INFO : accepted corpus with 15234 documents, 971 features, 1515531 non-zero entries


In [ ]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus_all, 
                                 id2word=dictionary_all, 
                                 time_slice=yearly_slices,
                                 num_topics=10,
                                 em_min_iter=1,
                                 em_max_iter=4)

2021-04-26 17:25:07,972 : INFO : using symmetric eta at 0.1
2021-04-26 17:25:07,973 : INFO : using serial LDA version on this node
2021-04-26 17:25:07,976 : INFO : running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 15234 documents, updating model once every 2000 documents, evaluating perplexity every 15234 documents, iterating 50x with a convergence threshold of 0.001000
2021-04-26 17:25:08,184 : INFO : PROGRESS: pass 0, at document #2000/15234
2021-04-26 17:25:12,213 : INFO : merging changes from 2000 documents into a model of 15234 documents
2021-04-26 17:25:12,232 : INFO : topic #6 (0.010): 0.006*"trade" + 0.006*"japan" + 0.006*"still" + 0.005*"united" + 0.005*"early" + 0.005*"close" + 0.005*"next" + 0.005*"closed" + 0.005*"lower" + 0.005*"three"
2021-04-26 17:25:12,233 : INFO : topic #1 (0.010): 0.016*"japanese" + 0.010*"diary" + 0.009*"currency" + 0.009*"tokyo" + 0.009*"forex" + 0.007*"lower" + 0.007*"around" + 0.007*"japan" + 0.007*"close" 